In [ ]:
from google.colab import files
files.upload()

Saving code_optimization_dataset.csv to code_optimization_dataset.csv


{'code_optimization_dataset.csv': b'unoptimized_code,optimized_code,optimization_type,complexity_change\r\n"def sort_numbers(nums):\n    for i in range(len(nums)):\n        for j in range(i + 1, len(nums)):\n            if nums[i] > nums[j]:\n                nums[i], nums[j] = nums[j], nums[i]\n    return nums","def sort_numbers(nums):\n    return sorted(nums)",Algorithmic Optimization,O(n^2) to O(n log n)\r\n"def find_duplicates(nums):\n    duplicates = []\n    for i in range(len(nums)):\n        for j in range(i + 1, len(nums)):\n            if nums[i] == nums[j] and nums[i] not in duplicates:\n                duplicates.append(nums[i])\n    return duplicates","def find_duplicates(nums):\n    seen = set()\n    duplicates = set()\n    for num in nums:\n        if num in seen:\n            duplicates.add(num)\n        else:\n            seen.add(num)\n    return list(duplicates)",Algorithmic Optimization,O(n^2) to O(n)\r\n"def fib(n):\n    if n <= 1:\n        return n\n    return fib(n

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
print(f"CUDA Available: {torch.cuda.is_available()}")

CUDA Available: True


In [ ]:
# Run this cell first
!pip install -q transformers accelerate bitsandbytes trl datasets peft sentencepiece safetensors pyngrok streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 140.3 MB/s eta 0:00:00


In [ ]:
DATA_PATH = "code_optimization_dataset.csv"

MODEL_NAME = "mistralai/Mistral-7B-v0.1"  
OUTPUT_DIR = "/content/drive/MyDrive/mistral-finetuned"
NGROK_AUTHTOKEN = "35t0D7y6l2yUBqAEjaI0nSNJVFk_67V8CLaVvQXr9ATPjpP5Y" 

In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
import random

df = pd.read_csv(DATA_PATH)
required_cols = {"unoptimized_code", "optimized_code"}
assert required_cols.issubset(df.columns), f"Dataset must contain columns: {required_cols}"

FileNotFoundError: [Errno 2] No such file or directory: 'code_optimization_dataset.csv'

In [ ]:
def make_pair(row):
    meta = []
    if 'optimization_type' in row and pd.notna(row['optimization_type']):
        meta.append(f"Type: {row['optimization_type']}")
    if 'complexity_change' in row and pd.notna(row['complexity_change']):
        meta.append(f"Complexity: {row['complexity_change']}")
    meta_str = " | ".join(meta)
    instruction = f"Optimize the following Python function/code for performance and clarity.\n\n"
    if meta_str:
        instruction += f"# {meta_str}\n\n"
    instruction += "### Unoptimized code:\n" + row['unoptimized_code'].strip()
    response = row['optimized_code'].strip()
    return {"instruction": instruction, "response": response}

pairs = [make_pair(r) for _, r in df.iterrows()]
random.shuffle(pairs)

In [ ]:
pairs = pairs[:3000]

split = int(len(pairs) * 0.9)
train_pairs = pairs[:split]
valid_pairs = pairs[split:]

train_ds = Dataset.from_list(train_pairs)
valid_ds = Dataset.from_list(valid_pairs)
dataset = DatasetDict({"train": train_ds, "validation": valid_ds})

print("Train size:", len(train_ds), "Validation size:", len(valid_ds))


Train size: 2700 Validation size: 300


In [ ]:
def format_for_sft(example):
    text = f"<s>[INST] {example['instruction']} [/INST] {example['response']}</s>"
    return {"text": text}

dataset = dataset.map(format_for_sft, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/2700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig

compute_dtype = "bfloat16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1 
)

model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470
None


In [ ]:
pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 11.5 MB/s eta 0:00:00


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    eval_strategy="steps",
    log_level="info",
    eval_steps=50,
    save_strategy="steps",
    save_steps=200,
    bf16=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    args=training_args,
)

Adding EOS to train dataset:   0%|          | 0/2700 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2700 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2700 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/300 [00:00<?, ? examples/s]

Using auto half precision backend


In [ ]:
pip install --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


In [ ]:
trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Saved fine-tuned model to ./{OUTPUT_DIR}")

The following columns in the Training set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2,700
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 338
  Number of trainable parameters = 3,407,872


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,0.219900,0.205316,0.242056,57877.000000,0.964032
100,0.069500,0.056461,0.084413,117289.000000,0.975389
150,0.052000,0.051393,0.071447,176355.000000,0.976445
200,0.051500,0.048081,0.064003,233812.000000,0.977706
250,0.054600,0.047970,0.064689,293349.000000,0.977334


The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 300
  Batch size = 2
The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 300
  Batch size = 2
The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 300
  Batch size = 2
The following columns in the Evaluation set don't have a corresponding

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,0.219900,0.205316,0.242056,57877.000000,0.964032
100,0.069500,0.056461,0.084413,117289.000000,0.975389
150,0.052000,0.051393,0.071447,176355.000000,0.976445
200,0.051500,0.048081,0.064003,233812.000000,0.977706
250,0.054600,0.047970,0.064689,293349.000000,0.977334
300,0.049600,0.046712,0.063897,351005.000000,0.976861


The following columns in the Evaluation set don't have a corresponding argument in `PeftModelForCausalLM.forward` and have been ignored: text. If text are not expected by `PeftModelForCausalLM.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 300
  Batch size = 2
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Saving model checkpoint to /content/drive/MyDrive/mistral-finetuned/checkpoint-338
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/27d67f1b5f57dc0953326b2601d68371d40ea8da/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "dtype": "bfloat16",
  "eos_token_id": 2,
  "head_dim": null,
  "hidden_act": "silu",
  "hidden_s

Saved fine-tuned model to .//content/drive/MyDrive/mistral-finetuned


In [ ]:
import torch
from transformers import pipeline
device = 0 if torch.cuda.is_available() else -1
pipe = pipeline("text-generation", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR, device=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
def infer_one(instr):
    prompt = f"<s>[INST] {instr} [/INST]"
    out = pipe(prompt, max_new_tokens=512, do_sample=False)[0]["generated_text"]
    prefix = f"<s>[INST] {instr} [/INST]"
    if out.startswith(prefix):
        return out[len(prefix):].strip()
    return out

In [ ]:
for i in range(min(3, len(valid_pairs))):
    sample = valid_pairs[i]
    print("\n--- SAMPLE", i+1, "---")
    print("INSTRUCTION (truncated):\n", sample['instruction'][:400], "...\n")
    print("EXPECTED OPTIMIZED:\n", sample['response'][:400], "...\n")
    print("MODEL OUTPUT:\n", infer_one(sample['instruction'])[:800], "...\n")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- SAMPLE 1 ---
INSTRUCTION (truncated):
 Optimize the following Python function/code for performance and clarity.

# Type: Algorithmic Optimization | Complexity: O(2^n) to O(n)

### Unoptimized code:
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2) ...

EXPECTED OPTIMIZED:
 def fibonacci(n, memo=None):
    if memo is None:
        memo = {}
    if n in memo:
        return memo[n]
    if n <= 1:
        memo[n] = n
        return n
    memo[n] = fibonacci(n - 1, memo) + fibonacci(n - 2, memo)
    return memo[n] ...

MODEL OUTPUT:
 def fibonacci(n, memo=None):
    if memo is None:
        memo = {}
    if n in memo:
        return memo[n]
    if n <= 1:
        memo[n] = n
    else:
        memo[n] = fibonacci(n - 1, memo) + fibonacci(n - 2, memo)
    return memo[n] ...


--- SAMPLE 2 ---
INSTRUCTION (truncated):
 Optimize the following Python function/code for performance and clarity.

# Type: Data Structure Optimization | Complexity: 